## Imports and settings

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import itertools
from tqdm import tqdm
from datetime import datetime
import os

In [ ]:
from generate_games import DelegationGame, flatgame_to_game, generate_delegation_games, generate_delegation_games_with_alignment_bounds
from evaluate_games import SimpleStat, get_stat_nash
import regret_bounds

## Plotting helpers

In [ ]:
def render_simple_bound_surface(welfare_regret:pd.Series, total_misalignment:pd.Series, bound_fn:regret_bounds.SimpleBound):
    x = np.linspace(0, welfare_regret.max(), num=50)
    y = np.linspace(0, total_misalignment.max(), num=50)
    xx, yy = np.meshgrid(x, y)
    zz = bound_fn(xx, yy)
    return xx, yy, zz

### Pyplot

In [ ]:
plt.rcParams['figure.dpi'] = 200 # 100

In [ ]:
def plot_stats(stats, bound_fn, view=(10, 30), ax=None, label=None, cmap=None):
    ax = plt.subplot(projection='3d') if ax is None else ax
    ax.view_init(*view)

    err = -stats.principals_welfare_regret if bound_fn is None else bound_fn(stats.welfare_regret, stats.total_misalignment) - stats.principals_welfare_regret

    ax.scatter(
        stats.welfare_regret,
        stats.total_misalignment,
        stats.principals_welfare_regret,
        c=err,
        cmap='Greens_r' if cmap is None else cmap,
        s=0.5,
        label=label)

    if bound_fn is not None:
        xx, yy, zz = render_simple_bound_surface(stats.welfare_regret, stats.total_misalignment, bound_fn)
        ax.plot_surface(xx, yy, zz, alpha=0.8)

    ax.set_xlabel('$\mathrm{WR}$')
    ax.set_ylabel('$\sum d_A$')
    ax.set_zlabel('$\hat\mathrm{WR}$')

In [ ]:
def plot_stats_multiview(stats, bound_fn):
    axl = plt.subplot(1, 2, 1, projection='3d')
    axr = plt.subplot(1, 2, 2, projection='3d')
    plot_stats(stats, bound_fn=bound_fn, view=(20, 10), ax=axl) # (20, 10) bound surface top and behind
    plot_stats(stats, bound_fn=bound_fn, view=(20, -40), ax=axr) # (20, -40) bound surface at front and top

### Plotly

In [ ]:
MUTED_BLUE='#1f77b4'
def fixed_colorscale(c): return [(0, c), (1, c)]

In [ ]:
layout_spec = dict(showlegend=False, margin=dict(l=0, r=0, b=0, t=0))
axis_spec = dict(title_font_size=10, dtick=1, tickfont_size=10)
axes_spec = dict(
    xaxis=axis_spec,
    yaxis=axis_spec,
    zaxis=axis_spec,
    xaxis_title_text='Cooperation failure',
    yaxis_title_text='Total agent misalignment',
    zaxis_title_text='Principals\' welfare regret',
)
aspect_spec = dict(x=1, y=1, z=0.8)
scene_spec = dict(
    aspectratio=aspect_spec,
    # camera_projection_type='orthographic', # optional render choice
    **axes_spec,
)
lighting_spec = dict(
    diffuse=1,
    fresnel=5,
    ambient=0.5,
    roughness=1.,
    specular=0.5,
)

In [ ]:
def make_simple_figure(stats, bound_fn):
    err = -stats.principals_welfare_regret if bound_fn is None else bound_fn(stats.welfare_regret, stats.total_misalignment) - stats.principals_welfare_regret

    fig = make_subplots(specs=[[{'is_3d':True}]])

    scatter = go.Scatter3d(
        x=stats.welfare_regret,
        y=stats.total_misalignment,
        z=stats.principals_welfare_regret,
        mode='markers',
        marker=dict(
            color=err,
            colorscale='Greens_r',
            size=1.5
        ))
    
    fig.append_trace(scatter, row=1, col=1)

    if bound_fn is not None:
        xx, yy, zz = render_simple_bound_surface(stats.welfare_regret, stats.total_misalignment, bound_fn)
        surface = go.Surface(x=xx, y=yy, z=zz, opacity=0.8, colorscale=fixed_colorscale(MUTED_BLUE), showscale=False, lighting=lighting_spec)
        fig.append_trace(surface, row=1, col=1)

    fig.update_layout(scene=scene_spec, **layout_spec)
    
    return fig

In [ ]:
def make_multiview_simple_figure(stats, bound_fn):
    fig1 = make_simple_figure(stats, bound_fn)
    fig2 = go.Figure(fig1)
    fig = make_subplots(1, 2, specs=[[{'is_3d':True}, {'is_3d':True}]], horizontal_spacing=0)
    fig.add_traces(data=fig1.data, rows=1, cols=1)
    fig.add_traces(data=fig2.data, rows=1, cols=2)
    fig.update_layout(
        scene=scene_spec,
        scene2=scene_spec,
        scene_camera_eye=dict(x=1.8, y=-1.5, z=.8),
        scene2_camera_eye=dict(x=2.2, y=.75, z=.8),
        **layout_spec,
        )
    return fig

## Data generation

Set up to generate configurable number of games and calculate stats, alternatively to read in stats from previous run

In [ ]:
READ_DATA = False # Falsey value means generate afresh and save; YYYY-mm-dd string means reread from that date
todaystr = READ_DATA if READ_DATA else datetime.today().strftime('%Y-%m-%d')

In [ ]:
n_games = 100000
SIMPLE_IDENT = f'simple_{n_games//1000}k'
SIMPLE_LOG_PATH = f'logs/{todaystr}/{SIMPLE_IDENT}/'
print(SIMPLE_LOG_PATH)

In [ ]:
if READ_DATA:
    stats = pd.read_csv(SIMPLE_LOG_PATH + 'data.csv', index_col=0)
else:
    stats = pd.DataFrame(tqdm(
        itertools.islice(map(get_stat_nash, generate_delegation_games()), n_games),
        total=n_games))
    os.makedirs(SIMPLE_LOG_PATH)
    stats.to_csv(SIMPLE_LOG_PATH + 'data.csv')
stats

In [ ]:
n_games_quite_aligned = 20000
max_epic = 0.5
SIMPLE_COMBO_IDENT = f'{SIMPLE_IDENT}_combo_aligned_{max_epic}_{n_games_quite_aligned//1000}k'
SIMPLE_COMBO_LOG_PATH = f"logs/{todaystr}/{SIMPLE_COMBO_IDENT}/"
print(SIMPLE_COMBO_LOG_PATH)

In [ ]:
if READ_DATA:
    stats_quite_aligned = pd.read_csv(SIMPLE_COMBO_LOG_PATH + 'extra_data.csv', index_col=0)
else:
    stats_quite_aligned = pd.DataFrame(tqdm(
        itertools.islice(map(get_stat_nash, generate_delegation_games_with_alignment_bounds(max_epic=max_epic)), n_games_quite_aligned),
        total=n_games_quite_aligned))
    os.makedirs(SIMPLE_COMBO_LOG_PATH)
    stats_quite_aligned.to_csv(SIMPLE_COMBO_LOG_PATH + 'extra_data.csv')

In [ ]:
stats_combo = pd.concat([stats, stats_quite_aligned], ignore_index=True)
stats_combo

## Produce plots

Plotting principals' welfare regret against cooperation failure (agent WR) and total misalignment (agent d_A from principals).

Also plotting theoretical bounds.

In [ ]:
default_simple_bound = regret_bounds.bound_principals_welfare_regret_simple()
conjecture_simple_bound = regret_bounds.bound_principals_welfare_regret_simple(mul=2*np.sqrt(2))

### Pyplot

In [ ]:
plot_stats_multiview(stats_combo, default_simple_bound)
plt.savefig(SIMPLE_COMBO_LOG_PATH + f'{SIMPLE_COMBO_IDENT}_render_with_bound.png', bbox_inches='tight')

In [ ]:
plot_stats_multiview(stats_combo, conjecture_simple_bound)
plt.savefig(SIMPLE_COMBO_LOG_PATH + f'{SIMPLE_COMBO_IDENT}_render_with_bound_conjecture.png', bbox_inches='tight')

### Plotly

In [ ]:
fig = make_multiview_simple_figure(stats_combo, default_simple_bound)
fig.show()
# fig.write_image(SIMPLE_COMBO_LOG_PATH + f'{SIMPLE_COMBO_IDENT}_3d_render_with_bound.png')

In [ ]:
# conjecture, tighter bound supported by simulations
fig = make_multiview_simple_figure(stats_combo, conjecture_simple_bound)
fig.show()
# fig.write_image(SIMPLE_COMBO_LOG_PATH + f'{SIMPLE_COMBO_IDENT}_3d_render_with_bound_conjecture.png')